In [ ]:
import tensorflow
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers
#from tf.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator # not needed?
from sklearn.model_selection import train_test_split # not needed?
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Put Colab in the context of this challenge


# os.chdir allows you to change directories, like cd in the Terminal
os.chdir('/content/drive//MyDrive/Colab Notebooks/preprocessed_images/data')

In [ ]:
# Define input shape and number of classes
img_height, img_width = 512, 512
num_classes = 2

# Load the pre-trained ResNet50 model with ImageNet weights
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

"""# Freeze the pre-trained layers
for layer in resnet.layers:
    layer.trainable = False"""

"""# Add new fully-connected layers for binary classification
x = layers.Flatten()(resnet.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(rate=0.3)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(rate=0.3)
output = layers.Dense(1, activation='sigmoid')(x)"""

"""# Create the final model
model = Model(inputs=resnet.input, outputs=output)"""

"""# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])"""

In [ ]:
def set_nontrainable_layers(model):
    model.trainable = False
    return model

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer_1 = layers.Dense(256, activation='relu')
    dropout_layer_1 = layers.Dropout(rate=0.3)
    dense_layer_2 = layers.Dense(128, activation='relu')
    dropout_layer_2 = layers.Dropout(rate=0.3)
    prediction_layer = layers.Dense(1, activation='sigmoid')
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer_1,
        dropout_layer_1,
        dense_layer_2,
        dropout_layer_2,
        prediction_layer
    ])
    return model

In [ ]:
def load_model():
  model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
  return model

In [ ]:
from tensorflow.keras import optimizers
def build_model():
    model = load_model()
    model = add_last_layers(model)
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [ ]:
resnet = set_nontrainable_layers(resnet)

In [ ]:
resnet = add_last_layers(resnet)

In [ ]:
resnet = build_model()

In [ ]:
data_dir = '/content/drive//MyDrive/Colab Notebooks/preprocessed_images/data'
shape = (512, 512)

In [ ]:
train_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=shape,
  shuffle = True)

In [ ]:
val_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=shape,
  shuffle = True)

In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take((2*val_batches) // 3)
val_ds = val_ds.skip((2*val_batches) // 3)

In [ ]:
es = EarlyStopping(patience=2,restore_best_weights=True)

# Train the model
history = resnet.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10,
        verbose=1,
        callbacks=[es])

In [ ]:
train_ds.class_names